In [1]:
import pandas as pd
import numpy as np
import os, sys
from datetime import date
import json

pd.set_option("display.max_rows", 100, "display.max_columns", 100)

def p(s):
    print(s)
    
pd.DataFrame.len = pd.Index.len = lambda x: print(len(x))

In [3]:
if os.path.exists('data/data.csv.gz'):
    dfFull = pd.read_csv('data/data.csv.gz', compression='gzip',index_col=0)
elif os.path.exists('../data/data.csv.gz'):
    dfFull = pd.read_csv('../data/data.csv.gz', compression='gzip',index_col=0)
else:
    YEARS = np.arange(1999,date.today().year)
    dfFull = pd.DataFrame()
    for i in YEARS:
        sys.stdout.write(f'{i} \r'); sys.stdout.flush()
        iData = pd.read_csv('https://github.com/guga31bb/nflfastR-data/blob/master/data/' \
                             'play_by_play_' + str(i) + '.csv.gz?raw=True',
                             compression='gzip', low_memory=False)
        dfFull = dfFull.append(iData, sort=True)

    dfFull.reset_index(drop=True, inplace=True)
    dfFull.to_csv('../data/data.csv.gz', compression='gzip')

C:\Users\mrkawa\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (11,12,13,14,15,16,23,24,62,63,79,80,81,93,94,95,100,101,102,132,133,134,135,136,137,138,139,144,145,150,157,158,164,165,166,167,184,204,205,206,207,219,249,250,251,255,257,260,261,262,263,306,310) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
d = dfFull[dfFull.season==2019].reset_index(drop=True)
if not os.path.exists('../data/2019.csv.gz'):
    d.to_csv('../data/2019.csv.gz', compression='gzip')

In [5]:
featureInfo = pd.read_csv('features.csv').iloc[:,0:4]
featureInfo[featureInfo.Use==1]

,Field,Description,Use,Type
0,complete_pass,Binary indicator for if the pass was completed.,1.0,Binary
1,fumble,Binary indicator for if a fumble occurred.,1.0,Binary
2,incomplete_pass,Binary indicator for if the pass was incomplete.,1.0,Binary
3,interception,Binary indicator for if the pass was intercepted.,1.0,Binary
4,no_huddle,Binary indicator for whether or not the play w...,1.0,Binary
5,pass_attempt,Binary indicator for if the play was a pass at...,1.0,Binary
6,pass_touchdown,Binary indicator for if the play resulted in a...,1.0,Binary
7,qb_dropback,Binary indicator for whether or not the QB dro...,1.0,Binary
8,qb_hit,Binary indicator if the QB was hit on the play.,1.0,Binary
9,qb_scramble,Binary indicator for whether or not the QB scr...,1.0,Binary


In [6]:
idCols = ['passer_id','rusher_id','receiver_id','kickoff_returner_player_id','punt_returner_player_id']
valCols = ['season','game_id','yards_gained','qb_dropback','shotgun','no_huddle','qb_kneel','qb_spike','qb_scramble','air_yards','yards_after_catch','first_down_rush','first_down_pass','third_down_converted','third_down_failed','fourth_down_converted','fourth_down_failed','incomplete_pass','interception','fumble_forced','fumble_not_forced','fumble_out_of_bounds','safety','penalty','tackled_for_loss','fumble_lost','qb_hit','rush_attempt','pass_attempt','sack','touchdown','pass_touchdown','rush_touchdown','return_touchdown','return_yards','fumble','complete_pass']
notUsed = [col for col in d.columns if col not in idCols + valCols]
df = dfFull[idCols + valCols]

In [8]:
['season'] + featureInfo[(featureInfo.Use==1) & (featureInfo.Type != 'ID')].Field.tolist()

['season',
 'complete_pass',
 'fumble',
 'incomplete_pass',
 'interception',
 'no_huddle',
 'pass_attempt',
 'pass_touchdown',
 'qb_dropback',
 'qb_hit',
 'qb_scramble',
 'rush_attempt',
 'rush_touchdown',
 'sack',
 'safety',
 'shotgun',
 'tackled_for_loss',
 'touchdown',
 'air_yards',
 'yards_after_catch',
 'yards_gained',
 'play_type',
 'posteam_type']

In [10]:
idCols = featureInfo[(featureInfo.Use==1) & (featureInfo.Type == 'ID')].Field.tolist()
valCols = featureInfo[(featureInfo.Use==1) & (featureInfo.Type != 'ID')].Field.tolist() + ['season', 'game_id']
notUsed = [col for col in d.columns if col not in idCols + valCols]
df = dfFull[['season'] + [x for x in featureInfo[(featureInfo.Use==1)].Field]]

game = df.set_index(valCols).stack().reset_index().groupby(['season','game_id',0]).sum()
season = df.set_index(valCols).stack().reset_index().groupby(['season',0]).sum()

game.to_csv('../data/gameData.csv')
season.to_csv('../data/seasonData.csv')

In [ ]:
game = pd.read_csv('../data/gameData.csv')
season = pd.read_csv('../data/seasonData.csv')

In [5]:
game = game[game['season']>=2011]
season = season[season['season']>=2011]

In [7]:
season

,season,0,complete_pass,fumble,incomplete_pass,interception,no_huddle,pass_attempt,pass_touchdown,qb_dropback,qb_hit,qb_scramble,rush_attempt,rush_touchdown,sack,safety,shotgun,tackled_for_loss,touchdown,air_yards,yards_after_catch,yards_gained
61941,2011,1,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
61942,2011,35,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
61943,2011,36,0.0,4.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0
61944,2011,37,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
61945,2011,39,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111163,2019,32013030-2d30-3033-3537-3033c61679ba,16.0,0.0,12.0,1.0,1,30.0,3.0,30.0,2.0,0,0.0,0.0,0.0,0.0,27,0.0,3.0,469.0,12.0,224.0
111164,2019,32013030-2d30-3033-3537-3034604f1124,100.0,7.0,53.0,3.0,5,162.0,7.0,172.0,23.0,11,20.0,0.0,6.0,0.0,135,0.0,7.0,1065.0,550.0,1066.0
111165,2019,32013030-2d30-3033-3537-31306de02154,284.0,37.0,164.0,12.0,69,512.0,24.0,542.0,113.0,30,50.0,2.0,49.0,0.0,497,3.0,33.0,3689.0,1390.0,2897.0
111166,2019,32013030-2d30-3033-3537-313925bb7f56,69.0,4.0,28.0,4.0,4,102.0,3.0,102.0,6.0,1,20.0,3.0,0.0,0.0,68,0.0,6.0,762.0,558.0,1202.0
